In [1]:
import numpy as np
import geopandas as gpd
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

import gee_data
import preprocessing as pp

In [3]:
def construct_model(path: str) -> keras.src.models.sequential.Sequential:
    """building the model skeleton

    Args:
        path (str): path to weight model

    Returns:
        keras.src.models.sequential.Sequential: model
    """
    model_dense = tf.keras.Sequential([
        tf.keras.Input((312, )),
        Dense(312),
        Dense(156),
        Dropout(0.1),
        Dense(78),
        Dropout(0.1),
        Dense(39),
        Dense(11, activation=tf.nn.softmax)
    ])
    
    model_dense.summary()

    model_dense.load_weights(f'{path}')

    return model_dense

In [17]:
def transform_data(bands_series: list, date_series: list) -> list:
    """Transform data to use inside model

    Args:
        bands_series (list): bands timeserie
        date_series (list): date timeserie

    Returns:
        list: bands timeserie flatten
    """
    bands_to_predict = []
    for band_series in bands_series.T:
        month_ndvi = pp.month_ndvi(date_series, band_series)
        bands_to_predict.append(pp.correction(month_ndvi))

    bands_to_predict = np.array(bands_to_predict).flatten().reshape(1, -1)

    return bands_to_predict

In [20]:
def __main__():
    start_date = '2022-01-01'  # Date de début
    end_date = '2022-12-31'    # Date de fin
    directory = "shapefile_prediction"
    filename = "models/multicrop_classification.h5"

    crops = ['Autres céréales _ Plante à fibre _ Riz','Blé tendre','Maïs grain et ensilage',
             'Orge','Protéagineux _ leguminause à grain','Colza','Tournesol','Autres oléagineux',
             'Fourrage_Estives et landes_Prairies permanentes_Prairies temporaires',
             'Vergers_Vignes_Fruits à coque_Oliviers','Autres cultures industrielles_Légumes ou fleurs']
    
    shapefile = gpd.read_file(f'{directory}/Parcelle.shp')

    date_series, bands_series = gee_data.get_timeSeries_full_bands(shapefile, start_date, end_date)

    bands_to_predict = transform_data(bands_series, date_series)

    model = construct_model()

    prediction = np.argmax(model.predict(bands_to_predict, verbose=0))

    print(f'La culture est du {crops[prediction]}')

In [21]:
__main__()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 312)                 │          97,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 156)                 │          48,828 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 156)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 78)                  │          12,246 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 78)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 39)                  │           3,081 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 11)                  │             440 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 162,251 (633.79 KB)

 Trainable params: 162,251 (633.79 KB)

 Non-trainable params: 0 (0.00 B)

La culture est du Colza
